In [1]:
library(igraph)
library(dplyr)
library(tibble)


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘dplyr’


The following objects are masked from ‘package:igraph’:

    as_data_frame, groups, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tibble’


The following object is masked from ‘package:igraph’:

    as_data_frame




In [2]:
.libPaths()

[1] "/Users/mnajm/miniconda3/envs/r_venv_conda/lib/R/library"

# 1. CF network

## 1.1 Loading the CF network

In [3]:
CF_PPI_network.pruned.interactions <- 
  read.table(file = "../CFnetwork/data/kegg_diff_pathways_network/diff_kegg_pathways_with_CFTR_interactors_PPI_direct_pruned_interactions_df.txt",
             sep = "\t",
             header = T,
             check.names = F)

CF_PPI_network.pruned.nodes <- 
  read.table(file = "../CFnetwork/data/kegg_diff_pathways_network/diff_kegg_pathways_with_CFTR_interactors_PPI_direct_pruned_nodes_df.txt",
             sep = "\t",
             header = T,
             check.names = F)

## 1.2 As an igraph object

In [4]:
source("../CFnetwork/scripts/pathways_to_network/network_utils.R")

In [5]:
CF_PPI_network.pruned <- new("PPI_network",
                             interactions=CF_PPI_network.pruned.interactions,
                             nodes=CF_PPI_network.pruned.nodes)
CF_PPI_network.pruned.with_CFTR <- CF_PPI_network.pruned

## 1.3 Remove CFTR from the network and its indirect interactors

In [6]:
CFTR_indirect_interactors <- CF_PPI_network.pruned.nodes[which(is.na(CF_PPI_network.pruned.nodes$sum)),
                                                         "Symbol"]

CF_PPI_network.pruned@nodes <- CF_PPI_network.pruned@nodes[which(!CF_PPI_network.pruned@nodes$Symbol %in% c("CFTR", CFTR_indirect_interactors)),]
CF_PPI_network.pruned@interactions <- CF_PPI_network.pruned@interactions[which(!CF_PPI_network.pruned@interactions$genesymbol_source %in% c("CFTR", CFTR_indirect_interactors) &
                                                                                 !CF_PPI_network.pruned@interactions$genesymbol_target %in% c("CFTR", CFTR_indirect_interactors)),]

## 1.4 Binding interactions into two directed interactions in both directions

In [7]:
# Pb of binding interactions
## Non binding
CF_PPI_network.pruned.interactions.non_binding <- 
  CF_PPI_network.pruned@interactions[which(!(CF_PPI_network.pruned@interactions$effect %in% c("binding/association"))),]
CF_PPI_network.pruned.interactions.non_binding <- 
  CF_PPI_network.pruned.interactions.non_binding[,c("genesymbol_source",
                                                           "genesymbol_target")]
colnames(CF_PPI_network.pruned.interactions.non_binding) <- c("from", 
                                                                     "to")

# ## Dissociation
CF_PPI_network.pruned.interactions.binding <- 
  CF_PPI_network.pruned@interactions[which(CF_PPI_network.pruned@interactions$effect %in% c("binding/association")),]
# Both directions for binding interactions
## one direction
CF_PPI_network.pruned.interactions.binding.one_direction <- 
  CF_PPI_network.pruned.interactions.binding[,c("genesymbol_source",
                                                       "genesymbol_target")]
colnames(CF_PPI_network.pruned.interactions.binding.one_direction) <- c("from", 
                                                                               "to")
## other direction
CF_PPI_network.pruned.interactions.binding.other_direction <- 
  CF_PPI_network.pruned.interactions.binding[,c("genesymbol_target", 
                                                       "genesymbol_source")]
colnames(CF_PPI_network.pruned.interactions.binding.other_direction) <- c("from", 
                                                                                 "to")
## both directions
CF_PPI_network.pruned.interactions.binding.both_directions <- 
  rbind(CF_PPI_network.pruned.interactions.binding.one_direction,
        CF_PPI_network.pruned.interactions.binding.other_direction)
CF_PPI_network.pruned.for_igraph <- rbind(CF_PPI_network.pruned.interactions.non_binding,
                                       CF_PPI_network.pruned.interactions.binding.both_directions)

In [8]:
CF_PPI_network.pruned.igraph <- graph_from_data_frame(CF_PPI_network.pruned.for_igraph, 
                                                   directed=TRUE)

## 1.5 Interesting nodes

### 1.5.1 Sink nodes

In [9]:
sink_nodes.final.pathways.df <- read.table(file = "../CFnetwork/data/sink_nodes/CFnetwork_sink_nodes_to_pathways.txt",
                                          sep = "\t",
                                          header = T,
                                          na.strings = "",
                                          check.names = FALSE)
sink_nodes.final.pathways.df <- sink_nodes.final.pathways.df[order(sink_nodes.final.pathways.df$Endpoint_cat),]
sink_nodes.final.pathways.df$Symbol <- as.character(sink_nodes.final.pathways.df$Symbol)
sink_nodes <- as.character(sink_nodes.final.pathways.df$Symbol)

### 1.5.2 Source nodes

In [10]:
source_nodes <- c("TRADD",
                  "PRKACA",
                  "SYK",
                  "CSNK2A1",
                  "SRC",
                  "PLCB1",
                  "PLCB3",
                  "EZR")

# 2. Differentially methylated CpG sites between CF and NCF dataframe

In [11]:
dm_CF_NCF_sup_table3_1267CpG <- 
  read.table(file = "../CFMethylomeSysbio/dm_CpG_CF_NCF_Magalhaes_2018_sup_table_3.tsv",
             skip = 1,
             quote = "\"",
             sep = "\t",
             header = T,
             check.names = F)

dm_CF_NCF_sup_table3_1267CpG <- 
  dm_CF_NCF_sup_table3_1267CpG[order(dm_CF_NCF_sup_table3_1267CpG$`p-value`),]
dm_CF_NCF_sup_table3_1267CpG$CpG_rank <- 1:dim(dm_CF_NCF_sup_table3_1267CpG)[1]

## 2.1 Keeping only CpG sites in the body of the genes

In [12]:
dm_CF_NCF_sup_table3_1267CpG_onlyGenes <- 
  dm_CF_NCF_sup_table3_1267CpG[which(dm_CF_NCF_sup_table3_1267CpG$`Genomic location`=="Body"),]

dm_CF_NCF_sup_table3_1267CpG_onlyGenes$Gene_rank <- 1:dim(dm_CF_NCF_sup_table3_1267CpG_onlyGenes)[1]

## 2.2 Searching for dm genes in the CF network

In [13]:
dm_CF_NCF_sup_table3_1267CpG_Genes.list <- unique(dm_CF_NCF_sup_table3_1267CpG_onlyGenes$Gene)

dm_CF_NCF_Genes_in_CF_network.list <- 
  dm_CF_NCF_sup_table3_1267CpG_Genes.list[which(dm_CF_NCF_sup_table3_1267CpG_Genes.list %in% CF_PPI_network.pruned.nodes$Symbol)]

dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <-
  dm_CF_NCF_sup_table3_1267CpG[which(dm_CF_NCF_sup_table3_1267CpG$Gene %in% dm_CF_NCF_Genes_in_CF_network.list),]

In [14]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

,Probe ID,Gene,p-value,Median β CF patients,Median β Controls,Genomic location,Island status,*Enhancer,CpG_rank
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>
647,cg13433729,EZR,1.2e-06,"0,61","0,49",Body,Open sea,TRUE,2
104,cg01899581,DIAPH3,9.0e-05,"0,64","0,47",Body,N_Shelf,FALSE,53
552,cg11321921,NFATC1,5.5e-04,"0,78","0,91",Body,Island,FALSE,240
306,cg05687091,NFATC1,6.1e-04,"0,54","0,64",Body,Island,FALSE,257
1086,cg23752651,TNFRSF1A,9.5e-04,"0,58","0,70",Body,Open sea,FALSE,374
1007,cg22240998,ITPR1,1.1e-03,"0,60","0,50",Body,Open sea,TRUE,420
891,cg18984715,CBLB,1.2e-03,"0,79","0,67",Body,Open sea,TRUE,441
234,cg04221461,AKT3,2.2e-03,"0,46","0,56",Body,Open sea,FALSE,613
968,cg21070009,MAP2K1,4.1e-03,"0,48","0,35",Body,Open sea,TRUE,852


## 2.3 Network analysis

In [15]:
CF_PPI_network.pruned.dists <- distances(CF_PPI_network.pruned.igraph, mode = "out")

### 2.3.1 Distance to sink nodes

In [16]:
### DM gene to sink nodes
CF_PPI_network.pruned.dm_to_sink_nodes <- CF_PPI_network.pruned.dists[dm_CF_NCF_Genes_in_CF_network.list, sink_nodes]


In [17]:
CF_PPI_network.pruned.dm_to_sink_nodes

,CASP1,CASP3,CASP7,CYBA,CYBB,DNM1L,GABARAP,ACTN4,ARPC5,CFL1,⋯,IRF5,IRF7,IRF9,JUN,NFATC1,NFKB1,NFKB2,RELA,RELB,STAT1
EZR,Inf,Inf,Inf,Inf,Inf,Inf,Inf,7,Inf,7,⋯,Inf,Inf,Inf,5,Inf,4,5,3,4,Inf
DIAPH3,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,3,Inf,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf
NFATC1,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,⋯,Inf,Inf,Inf,Inf,0,Inf,Inf,Inf,Inf,Inf
TNFRSF1A,4,3,4,5,6,4,9,6,7,7,⋯,4,4,5,5,5,4,4,5,5,4
ITPR1,3,4,4,10,11,7,6,10,12,10,⋯,5,5,8,6,3,4,7,5,6,7
CBLB,6,5,6,5,6,6,9,6,7,6,⋯,5,5,5,3,4,4,5,5,6,4
AKT3,7,7,7,9,10,8,9,6,11,6,⋯,6,2,6,4,2,3,2,4,3,5
MAP2K1,Inf,Inf,Inf,Inf,Inf,Inf,Inf,4,Inf,4,⋯,Inf,Inf,Inf,2,Inf,8,9,7,8,Inf
PLCG2,4,5,5,6,7,6,7,5,8,5,⋯,6,5,6,3,2,4,5,5,6,5
BAIAP2,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,2,Inf,⋯,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf


In [18]:
sink_nodes_downstream_to_dm <- data.frame(apply(X = CF_PPI_network.pruned.dm_to_sink_nodes,
      MARGIN = 1,
      FUN=function(x){
        return(sum(!is.infinite(x)))
      }))
colnames(sink_nodes_downstream_to_dm) <- "dist_to_CF_network_sink_nodes"
sink_nodes_downstream_to_dm['Gene'] <- rownames(sink_nodes_downstream_to_dm)
rownames(sink_nodes_downstream_to_dm) <- NULL

In [19]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <- merge(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
                                                    sink_nodes_downstream_to_dm,
                                                    by = "Gene")

In [20]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median β CF patients,Median β Controls,Genomic location,Island status,*Enhancer,CpG_rank,dist_to_CF_network_sink_nodes
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>
AKT3,cg04221461,2.2e-03,"0,46","0,56",Body,Open sea,FALSE,613,35
BAIAP2,cg02026049,7.9e-03,"0,65","0,76",Body,N_Shelf,FALSE,1123,2
CBLB,cg18984715,1.2e-03,"0,79","0,67",Body,Open sea,TRUE,441,35
DIAPH3,cg01899581,9.0e-05,"0,64","0,47",Body,N_Shelf,FALSE,53,2
EZR,cg13433729,1.2e-06,"0,61","0,49",Body,Open sea,TRUE,2,15
ITPR1,cg22240998,1.1e-03,"0,60","0,50",Body,Open sea,TRUE,420,35
MAP2K1,cg21070009,4.1e-03,"0,48","0,35",Body,Open sea,TRUE,852,15
NFATC1,cg11321921,5.5e-04,"0,78","0,91",Body,Island,FALSE,240,1
NFATC1,cg05687091,6.1e-04,"0,54","0,64",Body,Island,FALSE,257,1


### 2.3.2 Distance to source nodes

In [21]:
### Source nodes to sink nodes
CF_PPI_network.pruned.dm_from_source_nodes <- data.frame(t(CF_PPI_network.pruned.dists[source_nodes, dm_CF_NCF_Genes_in_CF_network.list]))
colnames(CF_PPI_network.pruned.dm_from_source_nodes) <- paste("from_",
                                                              colnames(CF_PPI_network.pruned.dm_from_source_nodes),
                                                              sep = "")

In [22]:
rownames(CF_PPI_network.pruned.dm_from_source_nodes)

[1] "EZR"      "DIAPH3"   "NFATC1"   "TNFRSF1A" "ITPR1"    "CBLB"    
 [7] "AKT3"     "MAP2K1"   "PLCG2"    "BAIAP2"

In [23]:
CF_PPI_network.pruned.dm_from_source_nodes['Gene'] <- rownames(CF_PPI_network.pruned.dm_from_source_nodes)
rownames(CF_PPI_network.pruned.dm_from_source_nodes) <- NULL

In [24]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <- merge(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
                                                    CF_PPI_network.pruned.dm_from_source_nodes,
                                                    by = "Gene")
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median β CF patients,Median β Controls,Genomic location,Island status,*Enhancer,CpG_rank,dist_to_CF_network_sink_nodes,from_TRADD,from_PRKACA,from_SYK,from_CSNK2A1,from_SRC,from_PLCB1,from_PLCB3,from_EZR
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AKT3,cg04221461,2.2e-03,"0,46","0,56",Body,Open sea,FALSE,613,35,3,Inf,5,Inf,2,7,7,Inf
BAIAP2,cg02026049,7.9e-03,"0,65","0,76",Body,N_Shelf,FALSE,1123,2,5,Inf,6,Inf,4,9,9,Inf
CBLB,cg18984715,1.2e-03,"0,79","0,67",Body,Open sea,TRUE,441,35,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf
DIAPH3,cg01899581,9.0e-05,"0,64","0,47",Body,N_Shelf,FALSE,53,2,8,Inf,9,Inf,7,12,12,Inf
EZR,cg13433729,1.2e-06,"0,61","0,49",Body,Open sea,TRUE,2,15,7,6,6,Inf,5,5,5,0
ITPR1,cg22240998,1.1e-03,"0,60","0,50",Body,Open sea,TRUE,420,35,4,Inf,2,Inf,10,1,1,Inf
MAP2K1,cg21070009,4.1e-03,"0,48","0,35",Body,Open sea,TRUE,852,15,3,2,4,Inf,2,5,5,3
NFATC1,cg11321921,5.5e-04,"0,78","0,91",Body,Island,FALSE,240,1,5,Inf,3,Inf,4,4,4,Inf
NFATC1,cg05687091,6.1e-04,"0,54","0,64",Body,Island,FALSE,257,1,5,Inf,3,Inf,4,4,4,Inf


### 2.3.4 Betweenness centrality

In [25]:
CF_PPI_network.bc.df <- data.frame(betweenness(CF_PPI_network.pruned.igraph))
CF_PPI_network.bc.df$Symbol <- rownames(CF_PPI_network.bc.df)
rownames(CF_PPI_network.bc.df) <- NULL
colnames(CF_PPI_network.bc.df) <- c("BC.score", "Symbol")

In [26]:
CF_PPI_network.bc.df.dm_CF_NCF_df <- CF_PPI_network.bc.df[which(CF_PPI_network.bc.df$Symbol %in%  dm_CF_NCF_Genes_in_CF_network.list),]

In [27]:
CF_PPI_network.bc.df.dm_CF_NCF_df

,BC.score,Symbol
,<dbl>,<chr>
69,3177.4806,PLCG2
73,415.6667,TNFRSF1A
120,2090.0913,MAP2K1
137,0.0000,CBLB
149,602.8583,ITPR1
214,15.0000,DIAPH3
225,661.0000,BAIAP2
227,2771.8197,AKT3
238,271.5944,EZR


In [28]:
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <- merge(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
                                                    CF_PPI_network.bc.df.dm_CF_NCF_df,
                                                    by.x = "Gene",
                                                    by.y = "Symbol",
                                                   all.x = T)
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median β CF patients,Median β Controls,Genomic location,Island status,*Enhancer,CpG_rank,dist_to_CF_network_sink_nodes,from_TRADD,from_PRKACA,from_SYK,from_CSNK2A1,from_SRC,from_PLCB1,from_PLCB3,from_EZR,BC.score
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AKT3,cg04221461,2.2e-03,"0,46","0,56",Body,Open sea,FALSE,613,35,3,Inf,5,Inf,2,7,7,Inf,2771.8197
BAIAP2,cg02026049,7.9e-03,"0,65","0,76",Body,N_Shelf,FALSE,1123,2,5,Inf,6,Inf,4,9,9,Inf,661.0000
CBLB,cg18984715,1.2e-03,"0,79","0,67",Body,Open sea,TRUE,441,35,Inf,Inf,Inf,Inf,Inf,Inf,Inf,Inf,0.0000
DIAPH3,cg01899581,9.0e-05,"0,64","0,47",Body,N_Shelf,FALSE,53,2,8,Inf,9,Inf,7,12,12,Inf,15.0000
EZR,cg13433729,1.2e-06,"0,61","0,49",Body,Open sea,TRUE,2,15,7,6,6,Inf,5,5,5,0,271.5944
ITPR1,cg22240998,1.1e-03,"0,60","0,50",Body,Open sea,TRUE,420,35,4,Inf,2,Inf,10,1,1,Inf,602.8583
MAP2K1,cg21070009,4.1e-03,"0,48","0,35",Body,Open sea,TRUE,852,15,3,2,4,Inf,2,5,5,3,2090.0913
NFATC1,cg11321921,5.5e-04,"0,78","0,91",Body,Island,FALSE,240,1,5,Inf,3,Inf,4,4,4,Inf,0.0000
NFATC1,cg05687091,6.1e-04,"0,54","0,64",Body,Island,FALSE,257,1,5,Inf,3,Inf,4,4,4,Inf,0.0000


### 2.3.5 Pathways

In [30]:
dm_CF_NCF_Genes_in_CF_network.CF_network_df <- CF_PPI_network.pruned.nodes[which(CF_PPI_network.pruned.nodes$Symbol %in% dm_CF_NCF_Genes_in_CF_network.list),]

dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <- merge(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
                                                    CF_PPI_network.pruned.nodes[,1:18],
                                                    by.x = "Gene",
                                                    by.y = "Symbol",
                                                   all.x = T)
dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df

Gene,Probe ID,p-value,Median β CF patients,Median β Controls,Genomic location,Island status,*Enhancer,CpG_rank,dist_to_CF_network_sink_nodes,⋯,Osteoclast differentiation,Regulation of actin cytoskeleton,RIG-I-like receptor signaling pathway,T cell receptor signaling pathway,Th17 cell differentiation,TNF signaling pathway,Toll-like receptor signaling pathway,Viral protein interaction with cytokine and cytokine receptor,sum,pathway
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<chr>
AKT3,cg04221461,2.2e-03,"0,46","0,56",Body,Open sea,FALSE,613,35,⋯,1,0,0,1,0,1,1,0,6,NA
BAIAP2,cg02026049,7.9e-03,"0,65","0,76",Body,N_Shelf,FALSE,1123,2,⋯,0,1,0,0,0,0,0,0,1,Regulation of actin cytoskeleton
CBLB,cg18984715,1.2e-03,"0,79","0,67",Body,Open sea,TRUE,441,35,⋯,0,0,0,1,0,0,0,0,2,NA
DIAPH3,cg01899581,9.0e-05,"0,64","0,47",Body,N_Shelf,FALSE,53,2,⋯,0,1,0,0,0,0,0,0,1,Regulation of actin cytoskeleton
EZR,cg13433729,1.2e-06,"0,61","0,49",Body,Open sea,TRUE,2,15,⋯,0,1,0,0,0,0,0,0,1,Regulation of actin cytoskeleton
ITPR1,cg22240998,1.1e-03,"0,60","0,50",Body,Open sea,TRUE,420,35,⋯,0,0,0,0,0,0,0,0,3,NA
MAP2K1,cg21070009,4.1e-03,"0,48","0,35",Body,Open sea,TRUE,852,15,⋯,1,1,0,1,0,1,1,0,6,NA
NFATC1,cg11321921,5.5e-04,"0,78","0,91",Body,Island,FALSE,240,1,⋯,1,0,0,1,1,0,0,0,4,NA
NFATC1,cg05687091,6.1e-04,"0,54","0,64",Body,Island,FALSE,257,1,⋯,1,0,0,1,1,0,0,0,4,NA


In [31]:
write.table(dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df,
            file = "../CFMethylomeSysbio/dm_CpG_CF_NCF_in_CF_network_analysis_20240809.tsv",
            sep = "\t",
            row.names = F,
            quote = FALSE)

# 3. Differentially methylated CpG sites between severe and mild dataframe

In [88]:
dm_severe_mild_sup_table4_189CpG <- 
  read.table(file = "../CFMethylomeSysbio/dm_CpG_CF_severe_mild_Magalhaes_2018_sup_table_4.tsv",
             skip = 1,
             quote = "\"",
             sep = "\t",
             header = T,
             check.names = F)

dm_severe_mild_sup_table4_189CpG <- 
  dm_severe_mild_sup_table4_189CpG[order(dm_severe_mild_sup_table4_189CpG$`p-value`),]
dm_severe_mild_sup_table4_189CpG$CpG_rank <- 1:dim(dm_severe_mild_sup_table4_189CpG)[1]

## 3.1 Keeping only CpG sites in the body of the genes

In [89]:
dm_severe_mild_sup_table4_189CpG_onlyGenes <- 
  dm_severe_mild_sup_table4_189CpG[which(dm_severe_mild_sup_table4_189CpG$`Genomic location`=="Body"),]

dm_severe_mild_sup_table4_189CpG_onlyGenes$Gene_rank <- 1:dim(dm_severe_mild_sup_table4_189CpG_onlyGenes)[1]

# 3.2 Searching for dm genes in the CF network

In [91]:
dm_severe_mild_sup_table4_189CpG_Genes.list <- unique(dm_severe_mild_sup_table4_189CpG_onlyGenes$Gene)

dm_CF_NCF_Genes_in_CF_network.list <- 
  dm_severe_mild_sup_table4_189CpG_Genes.list[which(dm_severe_mild_sup_table4_189CpG_Genes.list %in% CF_PPI_network.pruned.nodes$Symbol)]

dm_CF_NCF_Genes_in_CF_network.CF_network_df <- 
  CF_PPI_network.lcc.node_type.nodes[which(CF_PPI_network.pruned.nodes$Symbol %in% dm_CF_NCF_Genes_in_CF_network.list),]

dm_CF_NCF_Genes_in_CF_network.dm_CF_NCF_df <-
  dm_severe_mild_sup_table4_189CpG[which(dm_severe_mild_sup_table4_189CpG$Gene %in% dm_CF_NCF_Genes_in_CF_network.list),]

ERROR: Error in eval(expr, envir, enclos): object 'CF_PPI_network.lcc.node_type.nodes' not found


In [92]:
dm_CF_NCF_Genes_in_CF_network.list

character(0)